In [1]:
#import libraries to be used
import re
import pandas as pd
import requests as requests
from bs4 import BeautifulSoup

In [2]:
# Request Indeed page and create Bs4 Object
def extract(page):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.87 Safari/537.36'}
    url = f"https://ca.indeed.com/jobs?q=Data+Analyst&l=Toronto&start={page}"
    data = requests.get(url,headers)
    soup = BeautifulSoup(data.content, 'html.parser')
    return soup

In [4]:
# Isolate each job posting, and extract title, salary & link from each
titleList = []
linkList = []
vaxxList = []
salaryList = []
indeed_link = 'https://ca.indeed.com'
def transform(soup):    
    a = soup.select("a[id*=job_]") + (soup.select("a[id*=sj_]")) #Isolate Job Posting Tags
    for i in a: #Iterate through tags
        title = i.select_one("span[title]").text 
        link = i['href']
        full_link = indeed_link + link
        try:
            salary = i.find(class_='salary-snippet')
            salary = salary.find('span').text
        except:
            salary = '' 
        Vaxxed = WithinLink(full_link)
        titleList.append(title)
        salaryList.append(salary)
        linkList.append(full_link)
        vaxxList.append(Vaxxed)
        
    return titleList, salaryList, linkList, vaxxList


In [5]:
#Extract string data from within job-posting
def WithinLink(full_link):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.87 Safari/537.36'}
    data = requests.get(full_link, headers)
    soup = BeautifulSoup(data.content, 'html.parser')
    Vaxxed = soup.body.findAll(text=re.compile('vaccinated'))
    if Vaxxed == []:
        Vaxxed = False
    else:
        Vaxxed = True
    return Vaxxed


In [6]:
for i in range(0,20,10): #Value of 0 indicates first page, value of 10, second page. A value of 20 will pull first three pages.
    c = extract(i)
    transform(c)


In [7]:
lst = list(zip(titleList,salaryList, linkList, vaxxList)) #Zip lists to create a nested/2D list
lst
df = pd.DataFrame(lst, columns = ['Title','Salary','Link','VaccineReq'])
df

,Title,Salary,Link,VaccineReq
0,"Data Analyst, Summer 2022 Student Opportunities",,https://ca.indeed.com/rc/clk?jk=2565e400c1fe9b...,True
1,Data Analyst (Remote in Canada),,https://ca.indeed.com/company/InteraXon/jobs/D...,False
2,Data Analyst NOC-2172,"$6,638 a month",https://ca.indeed.com/company/hbimsw.ca/jobs/D...,False
3,IT Data Analyst III,,https://ca.indeed.com/rc/clk?jk=9fe4e435d0ad62...,False
4,Data Analyst,$50 - $65 an hour,https://ca.indeed.com/company/www.dawninfotek....,False
5,Junior Data Analyst,"$45,000 a year",https://ca.indeed.com/company/Delvinia/jobs/Ju...,False
6,Jr. Data Analyst,"$40,000 - $45,000 a year",https://ca.indeed.com/company/Flex-Surveys/job...,False
7,Data Analyst,,https://ca.indeed.com/company/Clutch-Canada/jo...,True
8,Data Analyst IV,,https://ca.indeed.com/rc/clk?jk=ae6990950e671d...,False
9,Business Analyst,,https://ca.indeed.com/company/MSi-Corp/jobs/Bu...,False


In [ ]:
df.to_csv('IndeedScrape.csv', index=False) #Export to CSV